In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

In [2]:
# Lettura dataset

data = pd.read_csv("train.csv", sep=",")
data.head()


,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,Y
0,39,K1,77516,R1,13,Q1,P1,M1,caucasian,Male,2174,0,40,USA,0
1,50,K2,83311,R1,13,Q2,P2,M2,caucasian,Male,0,0,13,USA,0
2,38,K3,215646,R2,9,Q3,P3,M1,caucasian,Male,0,0,40,USA,0
3,53,K3,234721,R3,7,Q2,P3,M2,black,Male,0,0,40,USA,0
4,28,K3,338409,R1,13,Q2,P4,M3,black,Female,0,0,40,Cuba,0


In [44]:
data.isna().sum()    # check

F0        0
F1     1836
F2        0
F3        0
F4        0
F5        0
F6     1843
F7        0
F8        0
F9        0
F10       0
F11       0
F12       0
F13     583
Y         0
dtype: int64

todo:
- stampare valori unique per ogni fiuciur
- imputazione valori NaN
- scaling
- bilanciamento
- rimozione outliers

e dopo:
- algoritmo di classificazione con le dovute metriche

In [6]:
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
print(x.shape)
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 258, stratify = y)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

print(np.unique(y, return_counts=True)[1]/len(y))
print(np.unique(y_train, return_counts=True)[1]/len(y_train))
print(np.unique(y_test, return_counts=True)[1]/len(y_test))

(32561, 14)
(32561,)
(24420, 14)
(24420,)
(8141, 14)
(8141,)
[0.75919044 0.24080956]
[0.75917281 0.24082719]
[0.75924334 0.24075666]


In [9]:
# IMPUTAZIONE NAN

from sklearn.impute import SimpleImputer, KNNImputer

print("SimpleImputer")
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
x_tr_notnan = imp.fit_transform(x_train)
x_ts_notnan = imp.transform(x_test)
print(x_tr_notnan[pd.isna(x_train[:,0]),0][0:10])
print(x_ts_notnan[pd.isna(x_test[:,0]),0][0:10])

print(x_tr_notnan)
df = pd.DataFrame(x_tr_notnan, columns = ['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13'])
df.head(35)

dftest = pd.DataFrame(x_ts_notnan, columns = ['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13'])
dftest.head(35)
# print("IterativeImputer")
# imp = IterativeImputer(missing_values=np.nan, max_iter=10)
# x_tr_notnan = imp.fit_transform(x_train)
# x_ts_notnan = imp.transform(x_test)
# print(x_tr_notnan[np.isnan(x_train[:,0]),0][0:10])
# print(x_ts_notnan[np.isnan(x_test[:,0]),0][0:10])

# print("KNNImputer")
# imp = KNNImputer(missing_values=np.nan, n_neighbors=3, weights="uniform")
# x_tr_notnan = imp.fit_transform(x_train)
# x_ts_notnan = imp.transform(x_test)
# print(x_tr_notnan[np.isnan(x_train[:,0]),0][0:10])
# print(x_ts_notnan[np.isnan(x_test[:,0]),0][0:10])

SimpleImputer
[]
[]
[[5.00000e+01 3.00000e+00 2.37868e+05 ... 0.00000e+00 5.00000e+01
  1.00000e+00]
 [4.10000e+01 5.00000e+00 1.95258e+05 ... 0.00000e+00 4.00000e+01
  1.00000e+00]
 [5.20000e+01 3.00000e+00 3.01229e+05 ... 0.00000e+00 4.00000e+01
  1.00000e+00]
 ...
 [3.10000e+01 3.00000e+00 1.18901e+05 ... 0.00000e+00 4.50000e+01
  1.00000e+00]
 [2.90000e+01 3.00000e+00 1.59788e+05 ... 0.00000e+00 8.00000e+01
  1.00000e+00]
 [4.80000e+01 3.00000e+00 2.99291e+05 ... 0.00000e+00 4.40000e+01
  1.00000e+00]]


,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,49.0,3.0,187454.0,2.0,9.0,2.0,6.0,2.0,1.0,0.0,99999.0,0.0,65.0,1.0
1,33.0,3.0,243674.0,2.0,9.0,5.0,11.0,1.0,1.0,0.0,0.0,0.0,46.0,1.0
2,31.0,5.0,213307.0,2.0,9.0,1.0,1.0,4.0,1.0,1.0,0.0,0.0,40.0,1.0
3,49.0,3.0,34377.0,16.0,8.0,2.0,7.0,2.0,1.0,0.0,0.0,0.0,40.0,1.0
4,21.0,3.0,203003.0,13.0,6.0,2.0,7.0,2.0,1.0,0.0,0.0,0.0,40.0,1.0
5,27.0,3.0,200641.0,13.0,6.0,2.0,2.0,3.0,1.0,1.0,0.0,0.0,40.0,2.0
6,43.0,3.0,328570.0,6.0,10.0,3.0,10.0,5.0,2.0,1.0,0.0,0.0,38.0,1.0
7,34.0,3.0,174789.0,6.0,10.0,2.0,5.0,2.0,1.0,0.0,0.0,0.0,50.0,1.0
8,36.0,3.0,117312.0,7.0,12.0,3.0,11.0,1.0,1.0,1.0,0.0,0.0,60.0,1.0
9,27.0,3.0,181284.0,16.0,8.0,2.0,4.0,2.0,2.0,0.0,0.0,0.0,45.0,1.0


In [125]:
freq = data['F13'].value_counts() 
print(freq) 

USA                   29184
Mexico                  643
Philippines             198
Germany                 137
Canada                  121
Puerto-Rico             114
El-Salvador             106
India                   100
Cuba                     95
England                  90
Jamaica                  81
South                    80
China                    75
Italy                    73
Dominican-Republic       70
Vietnam                  67
Guatemala                64
Japan                    62
Poland                   60
Columbia                 59
Taiwan                   51
Haiti                    44
Iran                     43
Portugal                 37
Nicaragua                34
Peru                     31
France                   29
Greece                   29
Ecuador                  28
Ireland                  24
Hong                     20
Cambodia                 19
Caribbean                19
Thailand                 18
Laos                     18
Yugoslavia          

In [147]:
# map da KX a X
datacopy = pd.read_csv("train.csv", sep=",")

colF1 = ['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8']
colF3 = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16']
colF5 = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7']
colF6 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
colF7 = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
colF8 = ['caucasian', 'black', 'asian', 'american', 'other']
colF9 = ['Male', 'Female']
colF13 = ['USA','Mexico', 'Philippines', 'Germany', 'Canada', 'Puerto-Rico', 'El-Salvador', 'India', 'Cuba','England', 'Jamaica','South','China','Italy','Dominican-Republic','Vietnam','Guatemala','Japan','Poland','Columbia','Taiwan','Haiti','Iran','Portugal','Nicaragua','Peru','France','Greece','Ecuador','Ireland','Hong','Cambodia','Caribbean','Thailand','Laos','Yugoslavia','Honduras','Hungary','Scotland','Netherlands']

datacopy.F1.replace(colF1, [1, 2, 3, 4, 5, 6, 7, 8], inplace = True)
datacopy.F3.replace(colF3, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], inplace = True)
datacopy.F5.replace(colF5, [1, 2, 3, 4, 5, 6, 7], inplace = True)
datacopy.F6.replace(colF6, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace = True)
datacopy.F7.replace(colF7, [1, 2, 3, 4, 5, 6], inplace = True)
datacopy.F8.replace(colF8, [1, 2, 3, 4, 5], inplace = True)
datacopy.F9.replace(colF9, [0, 1], inplace = True)
datacopy.F13.replace(colF13, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], inplace = True)


In [3]:
# map da KX a X
# datacopy = pd.read_csv("train.csv", sep=",")

colF1 = ['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8']
colF3 = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12', 'R13', 'R14', 'R15', 'R16']
colF5 = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7']
colF6 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14']
colF7 = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
colF8 = ['caucasian', 'black', 'asian', 'american', 'other']
colF9 = ['Male', 'Female']
colF13 = ['USA','Mexico', 'Philippines', 'Germany', 'Canada', 'Puerto-Rico', 'El-Salvador', 'India', 'Cuba','England', 'Jamaica','South','China','Italy','Dominican-Republic','Vietnam','Guatemala','Japan','Poland','Columbia','Taiwan','Haiti','Iran','Portugal','Nicaragua','Peru','France','Greece','Ecuador','Ireland','Hong','Cambodia','Caribbean','Thailand','Laos','Yugoslavia','Honduras','Hungary','Scotland','Netherlands']

data.F1.replace(colF1, [1, 2, 3, 4, 5, 6, 7, 8], inplace = True)
data.F3.replace(colF3, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], inplace = True)
data.F5.replace(colF5, [1, 2, 3, 4, 5, 6, 7], inplace = True)
data.F6.replace(colF6, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], inplace = True)
data.F7.replace(colF7, [1, 2, 3, 4, 5, 6], inplace = True)
data.F8.replace(colF8, [1, 2, 3, 4, 5], inplace = True)
data.F9.replace(colF9, [0, 1], inplace = True)
data.F13.replace(colF13, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], inplace = True)


In [4]:
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
print(x.shape)
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 258, stratify = y)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

print(np.unique(y, return_counts=True)[1]/len(y))
print(np.unique(y_train, return_counts=True)[1]/len(y_train))
print(np.unique(y_test, return_counts=True)[1]/len(y_test))

(32561, 14)
(32561,)
(24420, 14)
(24420,)
(8141, 14)
(8141,)
[0.75919044 0.24080956]
[0.75917281 0.24082719]
[0.75924334 0.24075666]


In [149]:
print("KNNImputer")
imp = KNNImputer(missing_values=np.nan, n_neighbors=3, weights="uniform")
x_tr_notnan = imp.fit_transform(x_train)
x_ts_notnan = imp.transform(x_test)
print(x_tr_notnan[np.isnan(x_train[:,0]),0][0:10])
print(x_ts_notnan[np.isnan(x_test[:,0]),0][0:10])

print(x_tr_notnan)
df = pd.DataFrame(x_tr_notnan, columns = ['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13'])
df.head(35)

KNNImputer
[]
[]
[[5.00000e+01 3.00000e+00 2.37868e+05 ... 0.00000e+00 5.00000e+01
  1.00000e+00]
 [4.10000e+01 5.00000e+00 1.95258e+05 ... 0.00000e+00 4.00000e+01
  1.00000e+00]
 [5.20000e+01 3.00000e+00 3.01229e+05 ... 0.00000e+00 4.00000e+01
  1.00000e+00]
 ...
 [3.10000e+01 3.00000e+00 1.18901e+05 ... 0.00000e+00 4.50000e+01
  1.00000e+00]
 [2.90000e+01 3.00000e+00 1.59788e+05 ... 0.00000e+00 8.00000e+01
  1.00000e+00]
 [4.80000e+01 3.00000e+00 2.99291e+05 ... 0.00000e+00 4.40000e+01
  1.00000e+00]]


,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,50.0,3.000000,237868.0,4.0,14.0,2.0,2.000000,2.0,1.0,0.0,0.0,0.0,50.0,1.000000
1,41.0,5.000000,195258.0,6.0,10.0,2.0,12.000000,2.0,1.0,0.0,0.0,0.0,40.0,1.000000
2,52.0,3.000000,301229.0,8.0,11.0,5.0,6.000000,5.0,1.0,1.0,0.0,0.0,40.0,1.000000
3,28.0,3.000000,173649.0,2.0,9.0,1.0,5.000000,4.0,2.0,1.0,0.0,0.0,40.0,1.000000
4,27.0,3.000000,29904.0,1.0,13.0,1.0,11.000000,1.0,1.0,1.0,0.0,0.0,40.0,1.000000
5,20.0,3.000000,94057.0,6.0,10.0,1.0,5.000000,4.0,1.0,0.0,0.0,0.0,15.0,1.000000
6,47.0,3.000000,199832.0,2.0,9.0,2.0,7.000000,2.0,1.0,0.0,0.0,0.0,40.0,1.000000
7,57.0,3.000000,562558.0,2.0,9.0,3.0,1.000000,5.0,1.0,1.0,0.0,0.0,40.0,1.000000
8,19.0,4.333333,249147.0,2.0,9.0,1.0,5.666667,4.0,1.0,0.0,0.0,0.0,30.0,1.000000
9,50.0,2.000000,95577.0,2.0,9.0,2.0,5.000000,3.0,1.0,1.0,0.0,0.0,12.0,10.333333


In [9]:
df.head(35)

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,50,K3,237868,R4,14,Q2,P2,M2,caucasian,Male,0,0,50,USA
1,41,K5,195258,R6,10,Q2,P12,M2,caucasian,Male,0,0,40,USA
2,52,K3,301229,R8,11,Q5,P6,M5,caucasian,Female,0,0,40,USA
3,28,K3,173649,R2,9,Q1,P5,M4,black,Female,0,0,40,USA
4,27,K3,29904,R1,13,Q1,P11,M1,caucasian,Female,0,0,40,USA
5,20,K3,94057,R6,10,Q1,P5,M4,caucasian,Male,0,0,15,USA
6,47,K3,199832,R2,9,Q2,P7,M2,caucasian,Male,0,0,40,USA
7,57,K3,562558,R2,9,Q3,P1,M5,caucasian,Female,0,0,40,USA
8,19,K3,249147,R2,9,Q1,P4,M4,caucasian,Male,0,0,30,USA
9,50,K2,95577,R2,9,Q2,P5,M3,caucasian,Female,0,0,12,USA


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24420 entries, 0 to 24419
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   F0      24420 non-null  float64
 1   F1      24420 non-null  float64
 2   F2      24420 non-null  float64
 3   F3      24420 non-null  float64
 4   F4      24420 non-null  float64
 5   F5      24420 non-null  float64
 6   F6      24420 non-null  float64
 7   F7      24420 non-null  float64
 8   F8      24420 non-null  float64
 9   F9      24420 non-null  float64
 10  F10     24420 non-null  float64
 11  F11     24420 non-null  float64
 12  F12     24420 non-null  float64
 13  F13     24420 non-null  float64
dtypes: float64(14)
memory usage: 2.6 MB


In [136]:
df.head()

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,50.0,3.0,237868.0,4.0,14.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,50.0,1.0
1,41.0,5.0,195258.0,6.0,10.0,2.0,12.0,2.0,1.0,0.0,0.0,0.0,40.0,1.0
2,52.0,3.0,301229.0,8.0,11.0,5.0,6.0,5.0,1.0,1.0,0.0,0.0,40.0,1.0
3,28.0,3.0,173649.0,2.0,9.0,1.0,5.0,4.0,2.0,1.0,0.0,0.0,40.0,1.0
4,27.0,3.0,29904.0,1.0,13.0,1.0,11.0,1.0,1.0,1.0,0.0,0.0,40.0,1.0


In [6]:
OneHotF1 = pd.get_dummies(df.F1)
OneHotF3 = pd.get_dummies(df.F3)
OneHotF5 = pd.get_dummies(df.F5)
OneHotF6 = pd.get_dummies(df.F6)
OneHotF7 = pd.get_dummies(df.F7)
OneHotF8 = pd.get_dummies(df.F8)
OneHotF9 = pd.get_dummies(df.F9)
OneHotF13 = pd.get_dummies(df.F13)

for i in range (1, len(colF1)):
    OneHotF1 = OneHotF1.rename(columns = {i: colF1[i-1]})
for i in range (1, len(colF3)):
    OneHotF3 = OneHotF3.rename(columns = {i: colF3[i-1]})
for i in range (1, len(colF5)):
    OneHotF5 = OneHotF5.rename(columns = {i: colF5[i-1]})
for i in range (1, len(colF6)):
    OneHotF6 = OneHotF6.rename(columns = {i: colF6[i-1]})
for i in range (1, len(colF7)):
    OneHotF7 = OneHotF7.rename(columns = {i: colF7[i-1]})
for i in range (1, len(colF8)):
    OneHotF8 = OneHotF8.rename(columns = {i: colF8[i-1]})
for i in range (1, len(colF9)):
    OneHotF9 = OneHotF9.rename(columns = {i: colF9[i-1]})
for i in range (1, len(colF13)):
    OneHotF13 = OneHotF13.rename(columns = {i: colF13[i-1]})

newdf = pd.concat([df, OneHotF1, OneHotF3, OneHotF5, OneHotF6, OneHotF7, OneHotF8, OneHotF9, OneHotF13], axis = 1)

newdf = newdf.drop(["F1","F3","F5","F6","F7","F8","F9","F13"], axis = 1)

In [7]:
newdf.head()
with pd.option_context('display.max_rows', 35, 'display.max_columns', None):  # more options can be specified also
    print(newdf)

         F0        F2    F4  F10  F11   F12  K1  K2  K3  K4  K5  K6  K7  8.0  \
0      50.0  237868.0  14.0  0.0  0.0  50.0   0   0   1   0   0   0   0    0   
1      41.0  195258.0  10.0  0.0  0.0  40.0   0   0   0   0   1   0   0    0   
2      52.0  301229.0  11.0  0.0  0.0  40.0   0   0   1   0   0   0   0    0   
3      28.0  173649.0   9.0  0.0  0.0  40.0   0   0   1   0   0   0   0    0   
4      27.0   29904.0  13.0  0.0  0.0  40.0   0   0   1   0   0   0   0    0   
...     ...       ...   ...  ...  ...   ...  ..  ..  ..  ..  ..  ..  ..  ...   
24415  44.0  323627.0  13.0  0.0  0.0   5.0   0   0   1   0   0   0   0    0   
24416  45.0  152752.0  11.0  0.0  0.0   3.0   0   1   0   0   0   0   0    0   
24417  31.0  118901.0  13.0  0.0  0.0  45.0   0   0   1   0   0   0   0    0   
24418  29.0  159788.0  14.0  0.0  0.0  80.0   0   0   1   0   0   0   0    0   
24419  48.0  299291.0   9.0  0.0  0.0  44.0   0   0   1   0   0   0   0    0   

       R1  R2  R3  R4  R5  R6  R7  R8  

In [135]:
print("Standard Scaler")
scaler = StandardScaler()


Standard Scaler
